<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','GILLETTE','NESTLEIND','PGHH','SIEMENS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100
#stock_ids = ['M&M', 'MARUTI']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    stock_list = ['AUBANK', 'COLPAL', 'GLAND','WHIRLPOOL']
    
    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list

    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list) 

    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        #print(len(test_list))

    q = 0

    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   

    else:
        p = 6

    df_yoy = pd.DataFrame()
    #print(test_list)
    if 'TTM' in test_list:
        end_year = 'TTM'
    else:
        end_year = 'Mar 2023'
          
    s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

    df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
    df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
    df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

    df_yoy.set_index('Year', inplace=True)
    df_yoy = df_yoy.T
    df_yoy['Stock'] = stock_name.upper() + '.NS'
    #df_yoy
    df_analysis = pd.concat([df_analysis, df_yoy])
    
df_analysis

ADANIENT
ADANIGREEN
ADANIPORTS
ADANITRANS
APOLLOHOSP
ASIANPAINT
AUBANK
AUROPHARMA
AXISBANK
BAJAJ-AUTO
BAJAJHLDNG
BAJFINANCE
BAJAJFINSV
BANDHANBNK
BANKBARODA
BATAINDIA
BERGEPAINT
BHARTIARTL
BIOCON
BOSCHLTD
BPCL
BRITANNIA
CANBK
CHOLAFIN
CIPLA
COALINDIA
COLPAL
DABUR
DEEPAKNTR
DIVISLAB
DLF
DMART
DRREDDY
EICHERMOT
EMAMILTD
FEDERALBNK
GAIL
GLAND
GODREJCP
GODREJIND
GRASIM
HAVELLS
HCLTECH
HDFC
HDFCAMC
HDFCBANK
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDPETRO
HINDUNILVR
IBREALEST
ICICIBANK
ICICIGI
ICICIPRULI
IGL
INDIANB
INDIGO
INDIGOPNTS
INDUSINDBK
INDUSTOWER
INFY
IOC
IRCTC
ITC
JINDALSTEL
JSWSTEEL
JUBLFOOD
KOTAKBANK
LT
LTIM
LUPIN
M&M
MARICO
MARUTI
MCDOWELL-N
MOTILALOFS
MUTHOOTFIN
NAM-INDIA
NAUKRI
NATIONALUM
NETWORK18
NMDC
NTPC
ONGC
PAGEIND
PEL
PFIZER
PIDILITIND
PIIND
PNB
POWERGRID
PVR
RELIANCE
SAIL
SBICARD
SBILIFE
SBIN
SHREECEM
SUNPHARMA
SUZLON
TATACONSUM
TATAMOTORS
TATAPOWER
TATASTEEL
TCS
TECHM
TIPSINDLTD
TITAN
TORNTPHARM
ULTRACEMCO
UPL
VEDL
VOLTAS
WHIRLPOOL
WIPRO
YESBANK
ZEEL


Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,TTM,Stock,Mar 2023
Sales,"36,533","35,924","40,379","43,403","39,537","69,420","130,497",ADANIENT.NS,NaN
Profit,"1,079",487,459,"1,122","1,086",952,"2,551",ADANIENT.NS,NaN
Sales,502,"1,480","2,058","2,549","3,124","5,133",NaN,ADANIGREEN.NS,"7,792"
Profit,-184,-210,-585,-50,193,553,NaN,ADANIGREEN.NS,"1,367"
Sales,"8,439","11,323","10,925","11,873","12,550","15,934","18,900",ADANIPORTS.NS,NaN
...,...,...,...,...,...,...,...,...,...
Profit,"11,039","10,242","11,542","12,252","13,903","15,141",NaN,WIPRO.NS,"14,766"
Sales,"10,627","12,529","19,811","19,258","12,611","12,528",NaN,YESBANK.NS,"14,800"
Profit,"5,053","6,205","2,349","-20,838","-4,762","1,434",NaN,YESBANK.NS,981
Sales,"6,434","6,686","7,934","8,130","7,730","8,189","8,308",ZEEL.NS,NaN


In [17]:
#df_analysis.columns
cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'TTM']
#df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
for col in cols:
    df_analysis[col] = df_analysis[col].str.replace(',', '').astype(float)

In [18]:
print('TTM: ', df_analysis['TTM'].count()/2)
print('Mar 2023: ', df_analysis['Mar 2023'].count()/2)

TTM:  84.0
Mar 2023:  34.0


In [23]:
lastestq = list(df_analysis[~df_analysis['Mar 2023'].isnull()]['Stock'].unique())
mypf.loc[mypf['Stock'].isin(lastestq), 'LatestQ'] = 1

In [24]:
df_analysis['Latest']  = df_analysis[["TTM", "Mar 2023"]].max(axis=1)
df_analysis.drop([ "TTM", "Mar 2023"], axis=1, inplace=True)

In [25]:
df_analysis['Max'] = df_analysis.max(axis=1)
df_analysis['Diff%'] = round((df_analysis['Max'] - df_analysis['Latest'])*100/df_analysis['Max'],2)

<ipython-input-25-28ae342f7da0>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_analysis['Max'] = df_analysis.max(axis=1)


In [26]:
df_analysis.to_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv', index=False)

In [27]:
df_tmp = df_analysis[df_analysis['Diff%'] == 0]
tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
len(tmp[tmp['Stock']>1].index)

51

In [28]:
star_stocks = list(tmp[tmp['Stock']>1].index)
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1

In [29]:
#mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv', index=False)